# 11. Parklife

```{sidebar} Dataset
  * [Wellcome Collection](https://wellcomecollection.org)
  * [V&A](https://www.vam.ac.uk/)
```

World events have lead to an increase in the popularity and usage of local parks. But which is the best park to take a stroll in ? Let's look at the Royal Parks of London depicted in art and medical collections to answer this question comprehensively, from an artistic and scientific viewpoint.

First we need to list the major [Royal Parks](https://www.royalparks.org.uk/) (with apologies to Kensington Gardens, Brompton Cemetary and Victoria Tower Gardens that don't make the list) 

In [1]:
royal_parks = (
  "Hyde Park",
  "Richmond Park",
  "Bushy Park",
  "Green Park",
  "St James's Park",
  "Regent's Park",
  "Greenwich Park")

## Wellcome Collection

We can use the Wellcome [API](https://developers.wellcomecollection.org) to get matching objects.

In [3]:
import requests
import pandas as pd
from urllib.parse import quote

wellcome_royal_parks_df = pd.DataFrame()

for park in royal_parks:
  req = requests.get(f"https://api.wellcomecollection.org/catalogue/v2/works?query={quote(park)}&aggregations=workType")
  wellcome_park_json = req.json()
  park_objects = []
  for workType in wellcome_park_json['aggregations']['workType']['buckets']:
    park_objects.append([park, workType['data']['label'], workType['count']])
  park_objects_df = pd.DataFrame(park_objects, columns=['Park', 'Label', 'Count'])  
  wellcome_royal_parks_df = wellcome_royal_parks_df._append(park_objects_df)

wellcome_royal_parks_df.sample(5)

,Park,Label,Count
2,Green Park,Pictures,57
5,Regent's Park,Film,2
2,Richmond Park,Pictures,2
0,St James's Park,Books,121
3,Green Park,Ephemera,11


Now we have the data, we can generate the graph showing which park is the most popular at Wellcome.

In [5]:
import altair as alt

alt.Chart(wellcome_royal_parks_df).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y='Park',
    x='Count:Q',
    color='Label:N').configure_legend(
    columns=5,
    orient='right',
    clipHeight=20,
    symbolLimit=60).properties(
    height=250)

alt.Chart(...)

So, for Wellcome Collection, Green Park is the place to be. However, taking a look at some of the items in more detail, there might be an element of fuzzy search going on here, as not all the results are about the "Green Park" in London. But regardless of this, we can see Richmond Park is not very extensively represented in Wellcome's collections.

## V&A at the Park

Now to run the same query against the [V&A collections](https://collections.vam.ac.uk)

In [7]:
import requests
import pandas as pd
from urllib.parse import quote

royal_parks = (
  "Hyde Park",
  "Richmond Park",
  "Bushy Park",
  "Green Park",
  "St James's Park",
  "Regent's Park",
  "Greenwich Park")

vam_royal_parks_df = pd.DataFrame()
park_objects = None

for park in royal_parks:
  park_objects = pd.read_json(f'https://api.vam.ac.uk/v2/objects/clusters/object_type/search?q="{quote(park)}"')
  # Potentially remove some object types with very low counts to make the graphic more readable
#  park_objects.drop( park_objects[ park_objects['count'] < 5 ].index, inplace=True)
  park_objects['park'] = park
  vam_royal_parks_df = vam_royal_parks_df._append(park_objects)

In [8]:
import altair as alt

alt.Chart(vam_royal_parks_df).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y='park',
    x='count:Q',
    color='value:N').configure_legend(
    columns=5,
    orient='right',
    clipHeight=20,
    symbolLimit=60).properties(
    height=250)

alt.Chart(...)

A clear victory for [Hyde Park](https://collections.vam.ac.uk/search/?q=%22hyde%20park%22&images_exist=true&page=1) there (helped hugely by the [Great Exhibition](https://collections.vam.ac.uk/search/?q=great%20exhibition&images_exist=true&page=1)), and perhaps Bushy Park is a bit too exclusive to be drawn or painted. As with Wellcome, some of these items are not really related to the park, but just have the name in the title or description somewhere. A more precise count could be achieved by using a place filter instead, but not all the parks have identifiers, so a text search has to do for now.

## Conclusion

Further research is needed, probably involving picnics.

# Notebook Updates

## 24 June 2024

  * Update append to _append in Pandas DataFrame calls
  * Removed unneeded import of test data